# Initial Setup of Environment
  * importing required packages/libraries for:
      * graphical models
      * perform statistical tests

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow.compat.v2.feature_column as fc
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind
from sklearn import linear_model
from IPython.display import clear_output
from six.moves import urllib

# to import from Kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load and Clean Datasets


The datasets are from games played in the highest level of competitive play. More specifically, Master, GrandMaster, and Challenger. 

In perspective, these players make up **~0.083% of a player base of 115+ million**.

* Dataset Source: https://www.kaggle.com/gyejr95/league-of-legends-challenger-ranked-games2020/data#

In [ ]:
# load training and testing data 
df_train = pd.read_csv("/kaggle/input/league-of-legends-challenger-ranked-games2020/Challenger_Ranked_Games.csv")
df_test = pd.read_csv("/kaggle/input/league-of-legends-challenger-ranked-games2020/Master_Ranked_Games.csv")
df_train.head()

## Cleaning the Data
The initial dataset contains data for both teams (i.e. Blue and Red teams). I am only interested in the stats of one team - in this case, I chosen the blue team.

* Notes: 
    * It does not matter which team is analyzed because the same stats are recoreded both teams
    * Both teams have winning and losing games recorded

* Training data will be based on Challenger games - the highest level of competitve play

* Testing data will be contain Master games - the second highest level of competitive play

In [ ]:
# clean training data
df_train_clean = df_train.filter(like="blue")
gameDuration = df_train.pop("gameDuraton")
df_train_clean.insert(0, "gameDuration", gameDuration, True)
y_train = df_train_clean.pop("blueWins")
df_train_clean.head()

In [ ]:
# clean testing data
df_test_clean = df_train.filter(like="blue")
df_test_clean.drop(["blueFirstBlood", "blueTotalHeal"], axis = 1)
y_test = df_test_clean.pop("blueWins")
df_test_clean.head()

# Visualizing/Describing the Data

## Key Characteristics of the Dataset


  * Both won and lost games are macro focused - both seek to secure objectives
  * Overall, the differences in macro management is greater than the differences in micro management
    * i.e) differences in dragon,baron,tower, and inhibitors are greater than the differences in gold/creep score (CS)/wards placed/wards destroyed
  * Majority of the measured variables are moderately/heavily right skewed
    * Dataset will benefit from a transformation to move closer to a normal distribution


In [ ]:
# Separate winning and losing stats
win_stats = df_train_clean.loc[df_train["blueWins"] == 1]
loss_stats = df_train_clean.loc[df_train["blueWins"] == 0]

## Key Observations in Summary Statistics:


---
  * Note: the following observations were based on the averages
---


  * Towers:
    * **First tower destroyed** was priortized by a factor of **~2.0x** in won games than lost games
    * **Total towers destroyed** was priortized by a factor of **~3.3x** in won games than lost games


  * Inhibitors:
    * **First inhibtors destroyed** was priortized **~11.3x** in won games than lost games
    * **Total inhibtors destroyed** was priortized **~9.3x** in won games than lost games


  * Dragons/Barons:
    * **First dragon** was priortized by a factor of **~2.0x** in won games than lost games
    * Won games have slain **~7.2x** more dragons than lost games (**Total dragons slain**)
    * **First baron** was priortized by a factor of **~4.6x** in won games than lost games
    * **Total barons slain** was priortized by a factor of **~4.3x** in won games than lost games

  * Kills/Deaths:
    * **First blood** was priortized by a factor of **~1.5x** in won games than lost games
    * In won games, there is **~1.6x** more **team kills** than lost games
    * In won games, there is **~0.6x** less **team deaths** than lost games


  * Micro/Miscellaneous: 
    * In won games, there were **~1.04x** more **wards placed** than in lost games 
    * In won games, there were **~1.1x** more **wards destroyed** than in lost games
    * There was a difference of **~20%** in **gold** between won and lost games
    * There was a difference of **~5.5%** in **creep score (CS)** between won and lost games
    * There was a difference of **~1.2%** in **total damage** between won and lost games
    

In [ ]:
# summary stats of team that won
win_stats.describe()

In [ ]:
# summary stats of team that lost
loss_stats.describe()

## Key Observations in Graphs

  * Won games have prioritized objectives by at least 3.0x more than lost games
  * Majority of the variables are moderately skewed to the right
    * Calculated skews are within a range of (-0.1 - 1.5)
  * First dragon is prioritized more than first baron 
  * The percentage of securing objectives is greater in won games than in lost games





### Histograms:


  * Note: the following observations are **true for both won and lost games**

  * **Normal** distributed variables:
    * Total gold
    * Total minion kills
    * Total level
    * Average level 


  * **Right skewed** distributed variables:
    * Ward placed
    * Ward kills
    * Kills
    * Death
    * Assists
    * Champion damage dealt
    * Jungle minion kills
    * Total level
    * Killing spree
    * Total heal
    * Object damage dealt




In [ ]:
def draw_histograms(df, variables, n_rows, n_cols):
    fig=plt.figure(figsize=(15,15))
    for i, var_name in enumerate(variables):
        ax=fig.add_subplot(n_rows,n_cols,i+1)
        df[var_name].hist(bins=20,ax=ax)
        ax.set_title(var_name+" Distribution")
        plt.tight_layout()
    
    plt.show()

In [ ]:
numerical_col_names = [c for c in df_test_clean.columns if c.lower()[4:9] != "first"]
# numerical_col_names = [c for c in numerical_col_names if c not in ["blueDragonKills", "blueBaronKills", "blueTowerKills", "blueInhibitorKills"]]
numerical_stats = df_test_clean[numerical_col_names]
histogram_all = df_test_clean[numerical_col_names]
histogram_data_win = win_stats[numerical_col_names]
histogram_data_lose = loss_stats[numerical_col_names]
histogram_all.head()

In [ ]:
from matplotlib import pyplot
def compare_histograms(df1, df2, variables, n_rows, n_cols):
        fig=plt.figure(figsize=(15,15))
        for i, var_name in enumerate(variables):
            ax=fig.add_subplot(n_rows,n_cols,i+1)
            df1[var_name].hist(bins=20, ax=ax, label="Won") # for histogram
            df2[var_name].hist(bins=20, ax=ax, label="Lost") # for histogram
            ax.set_title(var_name+" Distribution")
            pyplot.legend(loc="upper right")
            plt.tight_layout()
        
        plt.show()

In [ ]:
compare_histograms(histogram_data_win, histogram_data_lose, histogram_data_win, 9, 2)

In [ ]:
draw_histograms(histogram_all, histogram_all.columns, 9, 2)

### QQ Plots


  * Right skewed variables, shown to have light left tails, and heavy right tails

  
  * Notable Variables:
    * Total minion kills
      * looks to be the most normally distributed variable out of all
    * Champion damage dealt/assists/ total heal 
      * will need/benefit most with a transformation to a normal distribution
    * Total level
      * Both left and right tails are heavy
      * Majority of variables distributed near median


In [ ]:
import pylab 
def draw_qqplots(df, variables, n_rows, n_cols):
    fig=plt.figure(figsize=(15,15))
    for i, var_name in enumerate(variables):
        
        ax=fig.add_subplot(n_rows,n_cols,i+1)
        stats.probplot(df[var_name], dist="norm", plot=pylab)
        # df[var_name].hist(bins=20,ax=ax)
        ax.set_title(var_name+" qq plot")
        plt.tight_layout()
    
    plt.show()

In [ ]:
draw_qqplots(histogram_all, histogram_all.columns, 9, 2)

### Pie Charts

---
  * Note: Pie charts were chosen to better visualize variables that were either binary, or had a narrow range of values
---


  * Including both win and lost games (i.e. all games):
    * Equal chance for both teams to get first blood
    * First dragon is prioritized greater than first baron
    * It is rare to observe 4+ dragons slain
    * There is an uniform distribution of tower kills


  * Notable Comparisons:
    * For binary variables, won games secure the objective more often than lost games
    * It is more likely to capture a total of 1 dragon in lost games than won games
    * The frequency of capturing 2+ dragons differs greatly between won and lost games
    * Tower kills in won games have a uniform distribution, whereas lost games have majority of its data split between: 0, 2, and 6
    * In both won and lost games, 0 barons captured most frequent


In [ ]:
piechart_col_names = [c for c in df_test_clean.columns if c.lower()[4:9] == "first"]
# piechart_col_names = piechart_col_names + ["blueDragonKills", "blueBaronKills", "blueTowerKills", "blueInhibitorKills"]
piechart_all = df_train_clean[piechart_col_names]
piechart_all.head()

In [ ]:
piechart_win = win_stats[piechart_col_names]
piechart_loss = loss_stats[piechart_col_names]

In [ ]:
def draw_piecharts(df, variables, n_rows, n_cols):
    fig=plt.figure(figsize=(15,15))
    for i, var_name in enumerate(variables):
        ax=fig.add_subplot(n_rows,n_cols,i+1)
        percentages = list(df[var_name].value_counts(normalize=True, sort=False) * 100)
        labels = sorted(df[var_name].unique())
        explode = ([0.02] * len(percentages))
        ax.pie(percentages,shadow=True, explode=explode, labels=labels, autopct='%1.1f%%')
        ax.set_title(var_name)

        ax.legend(
          title="# of occurances",
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1),
          labels=['%s, %1.1f %%' % (l, s) for l, s in zip(labels, percentages)]
          )
        plt.tight_layout()
    
    plt.show()

In [ ]:
draw_piecharts(piechart_all, piechart_all.columns, 3, 3)

In [ ]:
draw_piecharts(piechart_win, piechart_win.columns, 3 ,3)

In [ ]:
draw_piecharts(piechart_loss, piechart_loss.columns, 3 ,3)

## Further Skew Calculation


  * Skew calculation found here: **https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.skew.html**


  * **Majority of the variables have moderate skewness** based on calculation

  
  * Notable Variables:
    * Assist has most of its weight in the left tail
      * Also true for total heal
    * Total level has	most of its weight in the right tail

In [ ]:
histogram_all.skew() # from scipy.stats library

# Preparing the Data for Analysis

## Key Points of Data Preparation: 


  * By normalizing the data via Box-Cox transformation, severely skewed variables appear closer to a normal distribution
   * Confirmed by skewness calculation, histograms, and QQ plots


  * Total level, average level, and total gold had enormous interaction with other variables and have been removed


  * Killing spree and assists are indirect measures of total kills and were removed due to redundancy 


  * Variables for further analysis: 
    * Numerical Varaibles: 
        * blueWardPlaced
        * blueWardkills
        * blueKills
        * blueDeath
        * blueChampionDamageDealt
        * blueTotalMinionKills
        * blueJungleMinionKills
        * blueTotalHeal
        * blueObjectDamageDealt
    
    * Binary Variables:
        * blueFirstBlood
        * blueFirstTower
        * blueFirstBaron
        * blueFirstDragon
        * blueFirstInhibitor


## Normalizing and Standardizing Data

Normalizing and standardizing data is couple of the methods in preprocessing data.

> Standardizing solves the differences of scales between variables. This is a problem because **larger scales will have a larger influence** on for example, a multivariate analysis, when that factor may not be an important predictor. When all variables are measured on the same scale, each variable is equally important when under analysis. 

> Normalzing solves skewed data/heteroskedasticity. With skewed data, there is **larger variability in the skewed region**. This is a problem in linear regression because for the **effect of the variable on the result will be enhanced** for values in the skewed region

> By normalizing, the variability becomes constant and the effect of the variable is true for all values.

***
  > More information about:

* [Benfits of normalizing data](https://towardsai.net/p/data-science/how-when-and-why-should-you-normalize-standardize-rescale-your-data-3f083def38ff)


* [Benefits of normalizing data 2](https://blog.minitab.com/blog/applying-statistics-in-quality-projects/how-could-you-benefit-from-a-box-cox-transformation)


* [Skewness](https://help.gooddata.com/doc/en/reporting-and-dashboards/maql-analytical-query-language/maql-expression-reference/aggregation-functions/statistical-functions/predictive-statistical-use-cases/normality-testing-skewness-and-kurtosis#:~:text=As%20a%20general%20rule%20of,the%20distribution%20is%20approximately%20symmetric.)

  




In [ ]:
from sklearn import preprocessing
def normalize_data(df):
  df = df+1
  scaler = preprocessing.PowerTransformer(method="box-cox", standardize=True).fit(df)
  norm_df = pd.DataFrame(scaler.transform(df),columns=df.columns)
  return norm_df

  > Here the data was normalzied with a Box-Cox transformation - a power transformation. 

  > As a result, the both problems were resovled:
  1. all variables fit within a similar scale
   * larger scaled variables will not longer have a larger influence in a model

  2. variables are closer to a normal distribution
  
  
   More information about:
  * [Box-Cox transformations](http://onlinestatbook.com/2/transformations/box-cox.html)

  * [Box-Cox transforamtions using sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.power_transform.html)

In [ ]:
norm = normalize_data(histogram_all)
norm.head()

In [ ]:
norm.skew()

In [ ]:
draw_histograms(norm, norm.columns, 6, 3)

In [ ]:
draw_qqplots(norm, norm.columns, 6, 3)

## Testing for Multicollinearity

### What is Multicollinearity?

  > Multicollinearity is the interaction between independent variables. It can be **measured by calculating the variance inflation factor (VIF)** and **visualized with a correlation matrix**. 
  
  > Additionally, VIF scores are only applicable to **ordinary least square** models. Whereas, correlation matrices are applicable to a wider range of models.


  > The greater a VIF is, the greater correlation it has with other variables. As a guideline, VIF >= 10 is most problematic because the less reliable that variable is in predicting an outcome. 
  * More info about: 
    * [Variance inflation factors (VIFs)](https://www.displayr.com/variance-inflation-factors-vifs/#:~:text=The%20higher%20the%20value%2C%20the,being%20regarded%20as%20very%20high.&text=If%20one%20variable%20has%20a,must%20also%20have%20high%20VIFs.)
    * [VIF Wiki](https://en.wikipedia.org/wiki/Variance_inflation_factor)
    * [Corrrelation Wiki](https://en.wikipedia.org/wiki/Correlation_and_dependence#Correlation_matrices)



### Variance Inflation Factor
  
  >  

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

In [ ]:
vif_init = calc_vif(norm)
vif_init

  > Total level, average level, and total gold have an enormous VIF, this means these variables are highly associated with other variables.


  > These will be removed to reduce the VIF of other variables and we can get closer to the true VIF scores.

  > Additionally, killing spree and assist are **indirect measures of total kills and is redundent**. Therefore, **killing spree and assists were removed** from further analysis.

In [ ]:
norm2 = norm.drop(["blueTotalLevel", "blueAvgLevel", "blueTotalGold", "blueKillingSpree", "blueAssist"], axis=1)
vif_norm2 = calc_vif(norm2)
vif_norm2

In [ ]:
import seaborn as sn
corr_matrix_norm = norm2.corr()
mask = np.triu(corr_matrix_norm)
f, ax = plt.subplots(figsize=(15,15))
ax = sn.heatmap(corr_matrix_norm, annot=True, mask=mask, cmap="coolwarm", square=True)
plt.show()

  > The **lower the number in the correlation matrix, the lower interaction** between the two variables. 


  > Some notable interactions: 
  * Total minion kills - ward kills
  * Total minion kills - wards placed


  > It is not as clear to why these variables are associated with each other, usually in game, minion kills and ward kills/placed are seen as independent. It raises the question: **Why do minion kills associate with ward kills/placed?** 


# Modelling the Data

### Key points of Modelling the Data: 
  * **logistic modelling** is best to represent the data
  * the following variables **are statistically significant**, and are included in modelling the data:
    * Numerical Variables: 
      * blueDragonKills         
      * blueTowerKills          
      * blueWardPlaced           
      * blueWardkills            
      * blueKills                
      * blueDeath                
      * blueTotalMinionKills     
      * blueJungleMinionKills   
      * blueObjectDamageDealt

    * Binary Varaibles: 
      * blueFirstBlood         
      * blueFirstTower          
      * blueFirstBaron           
      * blueFirstInhibitor
    

  * these variables will be observed when predicting the win probability of a game

## Choosing A Model to Represent the Data

### Which Model is Best? 
  > Given that the outcome is binary (i.e.) 0 = lose, 1 = win), a **logistic regression model** is best. 

### Why not Linear Regression? 
  > Linear regression **assumes that the response variable is continuous**. For example, if we were to graph a curve where total gold is on the X-axis, and win or lose on the Y-axis, there would be a sigmoidal curve, an S-shaped curve. This curve certainly does not show a linear relationship, therefore, linear regression wouldn’t be a good model for the data. 

### What is Logistic Regression?

  > Logistic regression models the probability of a true or false outcome, in this case the outcome is win or lose. This model is **best for classification**, and can be given a threshold to differentiate a true from or false prediction. 
  
  > The model is built similar to a linear regression model, where the predictor variables can be nominal, continuous, and categorical,
 and with a log(odds) transformation, predicts which group the data point belongs to.



![Logistic Regression Curve](https://miro.medium.com/max/725/1*QY3CSyA4BzAU6sEPFwp9ZQ.png)

More information about:

  * [What is Logistic Regression?](https://www.youtube.com/watch?v=yIYKR4sgzI8)
  * [How Logistic Regression is used](https://www.statisticssolutions.com/what-is-logistic-regression/#:~:text=Like%20all%20regression%20analyses%2C%20the,or%20ratio%2Dlevel%20independent%20variables.)

## Identifying Significant Variables


### Important Terminology: 
#### P-Values:
  > P-values indicate if the variable has an effect on the outcome variable. In other words, we set that variable to a value of 0 - if it makes a difference in the outcome, then it has an effect on the model.
	
  > When the variables P-value > 0.05, it is removed because it does not make a difference in predicting the outcome. Variables with a P-value < 0.05, they are retained in the model because removing it changes the predicted outcome. 

#### Maximum Likelihood Estimation (MLE):
  > MLE determines the parameters for each variable in the model, where it maximizes the likelihood of fitting predicted value to the observed value. These values can be determined by iterating through values and picking the model that has the least amount of error.
	
  > This is similar to fitting data using least squares in linear regression, except the data is projecting on a log(odds) line to find the least squares, then it is translated back into probability values and log(likelihood). 

#### Pseudo R<sup>2</sup> Value:
  > R<sup>2</sup> measures how well the model fits with the current data. It ranges from 0 to 1, the closer it is to 1, the better the model represents the data. 
---
More information about:
  * [P-values](https://blog.minitab.com/blog/adventures-in-statistics-2/how-to-interpret-regression-analysis-results-p-values-and-coefficients#:~:text=The%20p%2Dvalue%20for%20each,can%20reject%20the%20null%20hypothesis.&text=Typically%2C%20you%20use%20the%20coefficient,keep%20in%20the%20regression%20model.)
  * [Maximum Likelihood Estimation (MLE)](https://towardsdatascience.com/probability-concepts-explained-maximum-likelihood-estimation-c7b4342fdbb1)
  * [Pseudo R<sup>2</sup> values](https://thestatsgeek.com/2014/02/08/r-squared-in-logistic-regression/)
  * [Video explaination of P-values and R<sup>2</sup> values](https://www.youtube.com/watch?v=xxFYro8QuXA)
  * [Video explaination of MLE](https://www.youtube.com/watch?v=BfKanl1aSG0)

## Creating A Logistic Model

  ### First Model
  
  * for the first model, all the variables from the clean dataset were included
  #### P-Values:
    * the following had p-values > 0.05 and will be removed: 
        * blueFirstTower
        * blueFirstBaron
        * blueWardkills
        * blueTotalHeal
        * blueFirstDragon
        * blueBaronKills
        * blueChampionDamageDealt
        * blueTotalMinionKills
        * blueTotalHeal
        * blueObjectDamageDealt
    * all other variables are significant to the model

  ### R<sup>2</sup> Value:
    * surprisingly, the first model has a high R<sup>2</sup> value of 0.8462



In [ ]:
norm_X_train = piechart_all.join(norm2) # join categorical with numerical variables
X = norm_X_train.to_numpy()
Y = y_train
model = sm.Logit(Y,X).fit()
predictions = model.predict(X)
print_model = model.summary(xname=list(norm_X_train.columns))
print(print_model)

### Refining the First Model
  
  * for the second model, all the variables are statistically significant ((i.e.) p-values < 0.05). 
  #### R<sup>2</sup> Value: 
    * R<sup>2</sup> value did not decrease drastically, supporting the notion that removing insignificant variables had no effect on the models fit of the data.

In [ ]:
logit_insig = ["blueFirstTower", "blueFirstBaron", "blueWardkills", "blueChampionDamageDealt", "blueTotalMinionKills", "blueTotalHeal", "blueObjectDamageDealt"]
X = norm_X_train.drop(list(logit_insig), axis = 1)
Y = y_train
model = sm.Logit(Y,X).fit()
predictions = model.predict(X)
print_model = model.summary()
print(print_model)

In [ ]:
# odd ratios of each predictor variable, with raw data

# getting the names of significant variables
sig_vars = []
for key, val in model.params.iteritems():
    sig_vars.append(key)

raw_num_model_df = piechart_all.join(histogram_all) # join categorical with numerical variables
X_raw = raw_num_model_df[sig_vars]
Y = y_train
raw_num_model = sm.Logit(Y,X_raw).fit()
np.exp(raw_num_model.params)

The odds ratio compares the odds of winning to the odds of losing for an increase of 1 unit.


#### Displaying Significant Stats 

  * Note: feel free to look into the significant stats in won and lost games
    * there were NaN values because the win and lost values in "blueWins" was joined with the win and lost groups.
    * those said NaN rows were removed for a cleaner view

In [ ]:
blue_sig_stats = norm_X_train

blue_sig_stats["blueWins"] = y_train
blue_sig_stats.head()

In [ ]:
blue_wins = blue_sig_stats.where(df_train["blueWins"] == 1)
blue_wins = blue_wins.dropna()
blue_wins.head()

In [ ]:
blue_loses = blue_sig_stats.where(df_train["blueWins"] == 0)
blue_loses = blue_loses.dropna()
blue_loses.head()

# Predicting Win Probability

### Key points of Predicting Win Probability: 
  * after testing the model with Master tier games, the model achieved an accuracy of 96%!
  * columns have been labelled as numerical or categorical for the tensorflow model


## Background Info About TensorFlow Modelling


### Using a Linear Classification Model to Predict Win Probability
  > The probability of winning can take on values from 0 to 1 and NOT of values of 0 and 1. In other words, the probability is a continuous outcome, and not a binary outcome.
		
  > In the previous section, a logistic model was chosen to model the data because the outcome was binary and more importantly, it was given i.e.) win or loss. Therefore the p-values that identify the significant variables, or in other words the variables that influence winning, are credible. 
  
  > These significant variables will be used to create a linear classification model to predict the win probability of a game.

### How the data needs to be formatted for TensorFlow
  > Depending on how the variable is measured and encoded, the numbers can have different meanings. For example a categorical variable such as color, can encode 1 for red, 2 for blue, etc., but 1 < 2 does not make sense. In comparison to a numerical variable such as heart rate, 160BPM > 150BPM does make sense

* Note: BPM = beats per minute 
	

  > Tensorflow has estimators that use a “feature column” system.Each variable must be specified as categorical, numerical, etc., such that the model will know how to interpret the data for each feature. 


### Creating the TensorFlow Pipeline
  > A pipeline feeds data to the model in parts called ‘batches’. Everytime a batch is fed into the model, the parameters update to better fit the data. In other words, the more data exposed to the model, the more optimization made to the model. 


## Formatting Data for Tensorflow

In [ ]:
SIG_VARS = sig_vars
SIG_BIN_CAT_VARS = [v for v in list(piechart_all.columns) if v in SIG_VARS]
SIG_NUM_VARS = [v for v in list(histogram_all.columns) if v in SIG_VARS]
ALL_SIG_VARS = SIG_BIN_CAT_VARS + SIG_NUM_VARS
print(SIG_BIN_CAT_VARS)
print(SIG_NUM_VARS)

In [ ]:
# INSIG_VARS = logit_insig
# SIG_BIN_CAT_VARS = [v for v in list(piechart_all.columns) if v not in INSIG_VARS]
# SIG_NUM_VARS = [v for v in list(histogram_all.columns) if v not in INSIG_VARS]
# ALL_SIG_VARS = SIG_BIN_CAT_VARS + SIG_NUM_VARS

 Instead of hard coding the significant variables, I looped over the groups of variables and only included the ones that are significant

In [ ]:
def format_test_data(df):
    df = df.head(26904)
    bin_data = df[SIG_BIN_CAT_VARS]
    # normalize numerical variables
    num_data = df[SIG_NUM_VARS]
    norm_num_data = normalize_data(num_data)
    win_loss_col = df["blueWins"]
    test_data = pd.concat([bin_data, norm_num_data, win_loss_col], axis=1)
    return test_data

In [ ]:
def format_train_data(df):
    df = df.head(26904)
    bin_data = df[SIG_BIN_CAT_VARS]
    # normalize numerical variables
    num_data = df[SIG_NUM_VARS]
    norm_num_data = normalize_data(num_data)
    test_data = pd.concat([bin_data, norm_num_data], axis=1)
    return test_data

Here I formatted the datasets by: 
  1. getting the significant variable columns
  2. normalizing only the numerical variables
  3. attaching the binary and categorical variables
    * these were not transformed
  4. attaching the results/labels, indicating if blue team won or lost
    * this was only done for the test data

In [ ]:
clean_train_data = format_train_data(df_train)
clean_test_data = format_test_data(df_test)
clean_test_data.head()

## Feature Engineering for TensorFlow Model

In [ ]:
feature_columns = []
y_test_clean = clean_test_data.pop("blueWins")

for feature_name in SIG_BIN_CAT_VARS:
  vocabulary = clean_train_data[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in SIG_NUM_VARS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

This is where the significant variables are labelled as categorical, or numerical
  * more info can be found in the background information in this section

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=100):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return (ds)  # return a batch of the dataset
  return input_function  # return a function object for use

train_input_fn = make_input_fn(clean_train_data, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(clean_test_data, y_test_clean, num_epochs=1, shuffle=False)


Here is the pipeline that supplies the TensorFlow model with data. It includes:

  * values of the features
  * corresponding labels
  * feeds 100 points at a time

In [ ]:
ds = make_input_fn(clean_train_data, y_train, batch_size=10)()
for feature_batch, label_batch in ds.take(1):
  print('Some feature keys:', list(feature_batch.keys()))
  print()
  print('A batch of blueDragonKills:', feature_batch['blueDragonKills'].numpy())
  print()
  print('A batch of Labels:', label_batch.numpy())

This is an example of what a single input from a batch looks like

## Testing the Model and Results

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns= feature_columns)
# We create a linear estimator by passing the feature columns we created earlier

The linear model is created using the feature columns specified previously

In [ ]:
linear_est.train(train_input_fn)  # train
result = linear_est.evaluate(eval_input_fn)  # get model metrics/stats by testing on testing data

clear_output()  # clears console output
print(result)  # the result variable is simply a dict of stats about our model

Surprisingly, the model has an **accuracy of ~96%**

# TensorFlow Model Learning Perfomance

### Key points of Predicting Win Probability: 
  * given the accuracy of the model, predicted probabilties have a bimodal distrbution, with peaks at 0 and 1. 

  * the model has a low false positive rate, and a high true positive rate
    * as shown with the logarithmic ROC curve

In [ ]:
pred_dicts = list(linear_est.predict(eval_input_fn))
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

probs.plot(kind='hist', bins=20, title='predicted probabilities')

In [ ]:
from sklearn.metrics import roc_curve
from matplotlib import pyplot as plt

fpr, tpr, _ = roc_curve(y_test_clean, probs)
plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.xlim(0,)
plt.ylim(0,)